In [100]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
from hyperopt.pyll import scope
from sklearn.model_selection import KFold

import warnings
warnings.filterwarnings("ignore")
import optuna
%matplotlib inline
plt.style.use('seaborn')

In [23]:
df = pd.read_csv('data/_train_sem09 (1).csv')
df.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


In [24]:
df.shape

(3751, 1777)

In [25]:
X = df.drop(columns='Activity')
y = df['Activity']

In [26]:
y.value_counts()

1    2034
0    1717
Name: Activity, dtype: int64

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 42)

In [28]:
print(f'Shape of train {X_train.shape[0]}')
print(f'Shape of test {X_test.shape[0]}')

Shape of train 3000
Shape of test 751


### Base model without optimized hyperparameters

#### Logistic Regression

In [29]:
log_reg = linear_model.LogisticRegression(max_iter= 1000)
log_reg.fit(X_train, y_train)
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)
print(f'Prediction on train f1 score: {round(metrics.f1_score(y_train, y_train_pred),2)}')
print(f'Prediction on test accuracy score {round(log_reg.score(X_test,y_test),2)}')
print(f'Prediction on test f1-score {round(metrics.f1_score(y_test, y_test_pred),2)}')

Prediction on train f1 score: 0.89
Prediction on test accuracy score 0.75
Prediction on test f1-score 0.79


#### Random Forest

In [30]:
rf_mod = ensemble.RandomForestClassifier(random_state= 42)
rf_mod.fit(X_train, y_train)
y_train_pred = rf_mod.predict(X_train)
y_test_pred = rf_mod.predict(X_test)
print(f'Prediction on train f1 score: {round(metrics.f1_score(y_train, y_train_pred),2)}')
print(f'Prediction on test accuracy score {round(log_reg.score(X_test,y_test),2)}')
print(f'Prediction on test f1-score {round(metrics.f1_score(y_test, y_test_pred),2)}')

Prediction on train f1 score: 1.0
Prediction on test accuracy score 0.75
Prediction on test f1-score 0.83


### Optimization of hyperparameters

#### GridSearchCV logistic Regression

In [37]:
param_grid = {'penalty':['l2', 'none'],
              'solver': ['sag','lbfgs','newton-cg'],
              
}
grid_search = GridSearchCV(
    estimator= linear_model.LogisticRegression(max_iter = 1000),
    param_grid= param_grid,
    cv = 5,
    n_jobs = -1
)

grid_search.fit(X_train, y_train)
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)
print(f'Grid Search CV best params: {grid_search.best_params_}')
print(f'Grid Search CV prediction on test accuracy score {round(grid_search.score(X_test,y_test),2)}')
print(f'Grid Search CV prediction on test f1-score {round(metrics.f1_score(y_test, y_test_pred),2)}')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://sciki

Grid Search CV best params: {'penalty': 'l2', 'solver': 'lbfgs'}
Grid Search CV prediction on test accuracy score 0.75
Grid Search CV prediction on test f1-score 0.79


Метрику с помощью GridSearch CV улучшить не удалось

### Randomized Search CV Logistic Regression

In [46]:
%%time

param_distributions = {'penalty': ['l1','l2'],
                       'solver': ['saga', 'liblinear'],
                       'C': [0.01, 0.3, 0.5, 0.7, 0.9]}
random_search = RandomizedSearchCV(
    estimator = linear_model.LogisticRegression(max_iter= 1000),
    param_distributions= param_distributions,
    cv = 5,
    n_iter = 60,
    n_jobs = -1
)
random_search.fit(X_train, y_train)
y_test_pred = random_search.predict(X_test)
print(f'Randomized Search CV (Logistic Regression model) prediction on test accuracy score {round(random_search.score(X_test,y_test),2)}')
print(f'Randomized Search CV prediction on test f1-score {round(metrics.f1_score(y_test, y_test_pred),2)}')
print(f'Randomized Search CV best params: {random_search.best_params_}')

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_search.py:306: UserWarning: The total space of parameters 20 is smaller than n_iter=60. Running 20 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages

Randomized Search CV (Logistic Regression model) prediction on test accuracy score 0.75
Randomized Search CV prediction on test f1-score 0.78
Randomized Search CV best params: {'solver': 'liblinear', 'penalty': 'l1', 'C': 0.9}
CPU times: user 828 ms, sys: 465 ms, total: 1.29 s
Wall time: 2min 56s


Метрику с помощью RandomizedSearch CV улучшить не удалось

### Grid Search CV Random Forest

In [17]:
%%time
param_grid_rf = {'min_samples_leaf': [int(x) for x in list(np.linspace(5,50,10))],
                      'max_depth': [int(x) for x in list(np.linspace(10,100,10))],
                      'criterion': ['entropy', 'gini']}
grid_search_rf = GridSearchCV(
    estimator = ensemble.RandomForestClassifier(random_state=42),
    param_grid = param_grid_rf,
    cv = 5,
    n_jobs = -1
)
grid_search_rf.fit(X_train, y_train)
y_test_pred = grid_search_rf.predict(X_test)
print(f'Grid Search CV prediction on test accuracy score {round(grid_search_rf.score(X_test,y_test),2)}')
print(f'Grid Search CV prediction on test f1-score {round(metrics.f1_score(y_test, y_test_pred),2)}')
print(f'Grid Search CV best params: {grid_search_rf.best_params_}')

Grid Search CV prediction on test accuracy score 0.81
Grid Search CV prediction on test f1-score 0.83
Grid Search CV best params: {'criterion': 'gini', 'max_depth': 30, 'min_samples_leaf': 5}
CPU times: user 2.78 s, sys: 597 ms, total: 3.38 s
Wall time: 1min 49s


### Randomized Search CV Random Forest

In [104]:
%%time
param_distributions = {'min_samples_leaf': [int(x) for x in list(np.linspace(5,50,10))],
                      'max_depth': [int(x) for x in list(np.linspace(10,100,10))],
                      'n_estimators':[int(x) for x in list(np.linspace(100, 1000, 10))],
                      'criterion': ['entropy', 'gini',]}
random_search = RandomizedSearchCV(
    estimator = ensemble.RandomForestClassifier(random_state=42),
    param_distributions= param_distributions,
    cv = 5,
    n_iter = 60,
    n_jobs = -1
)
random_search.fit(X_train, y_train)
y_test_pred = random_search.predict(X_test)
print(f'Randomized Search CV prediction on test accuracy score {round(random_search.score(X_test,y_test),2)}')
print(f'Randomized Search CV prediction on test f1-score {round(metrics.f1_score(y_test, y_test_pred),2)}')
print(f'Randomized Search CV best params: {random_search.best_params_}')

Randomized Search CV prediction on test accuracy score 0.81
Randomized Search CV prediction on test f1-score 0.84
Randomized Search CV best params: {'n_estimators': 900, 'min_samples_leaf': 5, 'max_depth': 50, 'criterion': 'entropy'}
CPU times: user 8.02 s, sys: 786 ms, total: 8.81 s
Wall time: 3min 10s


In [82]:
random_state = 42
space = {'min_samples_leaf': scope.int(hp.quniform('min_samples_leaf',5,30,1)),
         'max_depth':scope.int(hp.quniform('max_depth',10,300,10)),
         'n_estimators':scope.int(hp.quniform('n_estimators', 100, 1500,50)),
        #  'criterion': hp.choice('criterion',['gini', 'entropy'])
}

In [83]:
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state = random_state):
    # функция получает комбинацию гиперпараметров в "params"
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score


In [84]:
%%time

trials = Trials() # используется для логирования результатов
best_rf=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo = tpe.suggest,
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best_rf))

100%|██████████| 20/20 [03:48<00:00, 11.42s/trial, best loss: -0.804692282486924] 
Наилучшие значения гиперпараметров {'max_depth': 260.0, 'min_samples_leaf': 5.0, 'n_estimators': 850.0}
CPU times: user 1min 42s, sys: 1.85 s, total: 1min 44s
Wall time: 3min 48s


In [85]:
model_rf = ensemble.RandomForestClassifier(
    n_estimators=int(best_rf['n_estimators']),
    max_depth = int(best_rf['max_depth']),
    min_samples_leaf = best_rf['min_samples_leaf'],
    # criterion = best_rf['criterion'],
    random_state = 42
)

model_rf.fit(X_train, y_train)
y_train_pred = model_rf.predict(X_train)
y_test_pred = model_rf.predict(X_test)
print(f'W/Hyperopt prediction on train f1-score {round(metrics.f1_score(y_train, y_train_pred),2)}')
print(f'W/Hyperopt prediction on test f1-score {round(metrics.f1_score(y_test, y_test_pred),2)}')
print(f'W/Hyperopt prediction on test accuracy score {round(model_rf.score(X_test,y_test),2)}')

W/Hyperopt prediction on train f1-score 0.7
W/Hyperopt prediction on test f1-score 0.73
W/Hyperopt prediction on test accuracy score 0.58


### Optuna with cross-validation

### Random Forest

In [88]:
kfolds = KFold(n_splits=5, shuffle=True, random_state=random_state)
def tuning(obj, n_trials = 20):
    study = optuna.create_study(direction="maximize")
    study.optimize(obj, n_trials= n_trials)
    params = study.best_params
    best_score = study.best_value
    print(f"Best score: {best_score}")
    print(f"Optimized parameters: {params}")
    return params


In [87]:
def optuna_random_forest(trial):
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)
  criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          criterion= criterion,
                                          random_state=random_state)
  scores = cross_val_score(
        model, X_train, y_train, cv=kfolds, scoring="f1"
  )
  return scores.mean()
  # обучаем модель

In [89]:
randomforest_params = tuning(optuna_random_forest)

[I 2022-09-12 16:32:26,949] A new study created in memory with name: no-name-f0553ff1-5a5f-4843-84b3-3854ad4394f2
[I 2022-09-12 16:32:32,935] Trial 0 finished with value: 0.808248231672555 and parameters: {'n_estimators': 179, 'max_depth': 25, 'min_samples_leaf': 6, 'criterion': 'entropy'}. Best is trial 0 with value: 0.808248231672555.
[I 2022-09-12 16:32:37,931] Trial 1 finished with value: 0.792677236389857 and parameters: {'n_estimators': 189, 'max_depth': 13, 'min_samples_leaf': 9, 'criterion': 'entropy'}. Best is trial 0 with value: 0.808248231672555.
[I 2022-09-12 16:32:41,829] Trial 2 finished with value: 0.8102317770033665 and parameters: {'n_estimators': 111, 'max_depth': 19, 'min_samples_leaf': 2, 'criterion': 'entropy'}. Best is trial 2 with value: 0.8102317770033665.
[I 2022-09-12 16:32:46,029] Trial 3 finished with value: 0.7875204948001553 and parameters: {'n_estimators': 170, 'max_depth': 13, 'min_samples_leaf': 10, 'criterion': 'gini'}. Best is trial 2 with value: 0.81

Best score: 0.8136525235052965
Optimized parameters: {'n_estimators': 160, 'max_depth': 15, 'min_samples_leaf': 3, 'criterion': 'entropy'}


In [92]:
rf_model = ensemble.RandomForestClassifier(n_jobs=-1, random_state= 42, **randomforest_params)
rf_model.fit(X_train,y_train)
y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)
print('Optuna optimized, train f1 score {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('Optuna optimized, test f1 score {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Optuna optimized, train f1 score 0.96
Optuna optimized, test f1 score 0.84


Hyperparameters optimized by Optuna have improved our test scores but our f1-score on a train selectionhave dropped by .04 points

### Logistic Regression

In [98]:
def optuna_logistic_reg(trial):
    penalty = trial.suggest_categorical('penalty', ['l2','none'])
    solver = trial.suggest_categorical('solver', ['sag', 'lbfgs'])
    C = trial.suggest_float('C', 0.1, 1)
    model = linear_model.LogisticRegression(
        penalty= penalty,
        solver= solver,
        C= C,
        max_iter= 1000
    )
    scores = cross_val_score(model, X_train, y_train, cv= kfolds, scoring = 'f1')
    return scores.mean()

In [101]:
log_reg_params = tuning(optuna_logistic_reg)

[I 2022-09-12 17:01:03,882] A new study created in memory with name: no-name-3774f3f3-6fd7-46c8-b887-ae9bb255b4d3
[I 2022-09-12 17:01:17,243] Trial 0 finished with value: 0.780051626976395 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.8977883077263543}. Best is trial 0 with value: 0.780051626976395.
[I 2022-09-12 17:01:51,957] Trial 1 finished with value: 0.7809084573728369 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.8405129529035475}. Best is trial 1 with value: 0.7809084573728369.
[I 2022-09-12 17:02:24,165] Trial 2 finished with value: 0.7800611212039211 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.8571646454553068}. Best is trial 1 with value: 0.7809084573728369.
[I 2022-09-12 17:02:55,362] Trial 3 finished with value: 0.7208920269980985 and parameters: {'penalty': 'none', 'solver': 'lbfgs', 'C': 0.23118209862729744}. Best is trial 1 with value: 0.7809084573728369.
[I 2022-09-12 17:04:24,035] Trial 4 finished with value: 0.7552885739966833 a

Best score: 0.7899726081966426
Optimized parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.11050614306199202}


In [103]:
log_reg = linear_model.LogisticRegression(**log_reg_params, max_iter = 1000)
log_reg.fit(X_train,y_train)
y_train_pred = log_reg.predict(X_train)
y_test_pred = log_reg.predict(X_test)
print('Optuna optimized, train f1 score {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
print('Optuna optimized, test f1 score {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Optuna optimized, train f1 score 0.85
Optuna optimized, test f1 score 0.79


We did not improve our score with optuna hyperparameters optimization